In [1]:
# for reference see this code: /home/csutter/DRIVE/weather_img_concatmodels/cnn_hrrr_fcsthr2/nested_skill_streamline_vs_blending.ipynb

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)        # Use full width of the console
pd.set_option("display.max_colwidth", None)  # Show full content in each column

# CNN model selection

This file: /home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv 
- Has Baseline model performance appended for each of the 36 BL models (hyperparams L2 and dropout set to 0)
- Has HT model performance appended for each of the 64 HT models (16 hyperparam sets x top 4 BL models)
- For final model selection, consider all 36 (with initial HTs of 0s) + 64 (with other HTs tuned) for final selection - this is why all models were appended into the same csv file.

### CNN - Baseline selection (BL)

In [2]:
##### For Baseline top 4 model selections

df = pd.read_csv("/home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv")

# Important! If recreating the selection from the 36 baseline models, run this code below to subset just the first 36 runs. O/w if doing final selection post hyptuning, don't need to subset - want to consider all models. See markdown note above for more detail.
df = df.head(36)

df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)


top10_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:4]

display(selected4)

print(selected4["exp_details"])

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910


20        _A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue
2        _A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse
26         _A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue
4     _A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse
Name: exp_details, dtype: object


In [3]:
# TESTING 7/26 -  1) what percentile does it fall in for each. 2) what is the spread within that percentile (e.g. if all super similar, then the performance of that class doesnt matter as much, so weight its placement accordingly (i.e. if all models get between 70-71% on dry, then what does the top 99% model matter when it's just spread of 1%? 3) what do I care about them? Our model importances

df["adjacc"] = df["ok"]/df["totalims"]


overall_imp = 1.0
perc_snow_severe_imp = 1.0
perc_poor_viz_imp = 0.8
perc_snow_imp = 0.7
adjacc_imp = 0.2
perc_dry_imp = 0.5
perc_wet_imp = 0.5

# overall_imp = 1.0
# perc_snow_severe_imp = 1.0
# perc_poor_viz_imp = 0.9
# perc_snow_imp = 0.9
# adjacc_imp = 0.2
# perc_dry_imp = 0.8
# perc_wet_imp = 0.8


overall_std = df["overall"].std(axis=0)
perc_snow_severe_std = df["perc_snow_severe"].std(axis=0)
perc_poor_viz_std = df["perc_poor_viz"].std(axis=0)
perc_snow_std = df["perc_snow"].std(axis=0)
adjacc_std = df["adjacc"].std(axis=0)
perc_dry_std = df["perc_dry"].std(axis=0)
perc_wet_std = df["perc_wet"].std(axis=0)

std_list = [overall_std,perc_snow_severe_std,perc_poor_viz_std,perc_snow_std,adjacc_std,perc_dry_std,perc_wet_std]
# stdev weights alone are too big, want them to only range in scale between 0.8 and 1.0, o/w it will overpower the assigned weights. Rescale them

def rescale_list(data_list, new_min=0.8, new_max=1.0):
    # Handle empty list case
    if not data_list:
        return []

    # Calculate the minimum and maximum values of the original list
    original_min = min(data_list)
    original_max = max(data_list)

    # Handle the case where all values are the same to avoid division by zero
    if original_min == original_max:
        return [new_min] * len(data_list)

    # Apply the min-max scaling formula to each value
    rescaled_list = [
        ((x - original_min) / (original_max - original_min)) * (new_max - new_min) + new_min
        for x in data_list
    ]

    return rescaled_list

std_scaled = rescale_list(std_list, new_min=0.8, new_max=1.0)

overall_wt = overall_imp*std_scaled[0]
overperc_snow_severe_wt = perc_snow_severe_imp*std_scaled[1]
perc_poor_viz_wt = perc_poor_viz_imp*std_scaled[2]
perc_snow_wt = perc_snow_imp*std_scaled[3]
adjacc_wt = adjacc_imp*std_scaled[4]
dry_wt = perc_dry_imp*std_scaled[5]
perc_wet_wt = perc_wet_imp*std_scaled[6]

importances_new = overall_wt,overperc_snow_severe_wt,perc_poor_viz_wt,perc_snow_wt,adjacc_wt,dry_wt,perc_wet_wt

scale = sum(importances_new)

print(importances_new)

print(scale)

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importances_new"] = (values * importances_new).sum(axis=1) / scale
importances_df= df.sort_values("importances_new", ascending=False)[0:10]
display(importances_df)

# df['Row_STDEV'] = df[['A', 'B', 'C']].std(axis=1)

(0.8414411744117026, 0.9031586486879329, 0.7652876203677103, 0.7, 0.16000000000000003, 0.4227307113222218, 0.41039763787187866)
4.203015792661446


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,adjacc,importances_new
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910,0.898581,0.699674
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488,0.909275,0.698331
22,107445,72289,7635,5783,14945,9176,42290,30099,38000,23972,4575,3259,96364,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:33:33.641890,0.672800,0.630842,0.712350,0.613985,0.757433,0.711729,0.685268,0.896868,0.696503
28,107445,71027,7635,5791,14945,9061,42290,28954,38000,23838,4575,3383,95382,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:15:03.281649,0.661054,0.627316,0.739454,0.606290,0.758481,0.684654,0.683239,0.887729,0.694684
10,107445,71241,7635,5837,14945,9087,42290,29057,38000,24004,4575,3256,95638,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 04:08:26.195298,0.663046,0.631684,0.711694,0.608029,0.764506,0.687089,0.680600,0.890111,0.692380
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796,0.906482,0.691186
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098,0.909107,0.688335
23,107445,70746,7635,5779,14945,8490,42290,29357,38000,23759,4575,3361,96445,nestcv_5cat_twotrain,_A_densenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:43:05.127303,0.658439,0.625237,0.734645,0.568083,0.756909,0.694183,0.675811,0.897622,0.687682
8,107445,72760,7635,5625,14945,8915,42290,30671,38000,24421,4575,3128,97003,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 03:55:06.972875,0.677184,0.642658,0.683716,0.596521,0.736739,0.725254,0.676977,0.902815,0.687547
29,107445,69292,7635,5778,14945,8508,42290,28059,38000,23588,4575,3359,95733,nestcv_5cat_twotrain,_A_densenet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:24:54.035309,0.644907,0.620737,0.734208,0.569287,0.756778,0.663490,0.668900,0.890995,0.681363


In [49]:
# TESTING SOME OTHER VARIATIONS
print("AVERAGE WAY")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["avgmetrics"] = 0.25*df['overall']+0.75*df['avg_recall']
avgmetrics = df.sort_values("avgmetrics", ascending=False)[0:10]
display(avgmetrics)

print("IMPORTANCES")
importances = [2,1.5,1,1,1.5]
cols = ["perc_snow_severe", "perc_snow", "perc_wet", "perc_dry", "perc_poor_viz"]
values = df[cols].values
df["weighted_avg_recall"] = (values * importances).sum(axis=1) / sum(importances)
wtdavg = df.sort_values("weighted_avg_recall", ascending=False)[0:10]
display(wtdavg)

print("IMPORTANCE + AVG")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["impAndAvg"] = (1/3)*df['overall']+(1/3)*df['avg_recall']+(1/3)*df["weighted_avg_recall"] 
impAndAvg = df.sort_values("impAndAvg", ascending=False)[0:10]
display(impAndAvg)

print("IMPORTANCES OLD PAPER")
importancesOld =  [10, 10, 8, 8, 2, 5, 5]
df["adjacc"] = df["ok"]/df["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)
importanceold = df.sort_values("importanceold", ascending=False)[0:10]
display(importanceold)

AVERAGE WAY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,adjacc,overall_PercentileRank,overall_stdev,perc_snow_severe_PercentileRank,perc_snow_severe_stdev,perc_poor_viz_PercentileRank,perc_poor_viz_stdev,perc_snow_PercentileRank,perc_snow_stdev,adjacc_PercentileRank,adjacc_stdev,perc_dry_PercentileRank,perc_dry_stdev,perc_wet_PercentileRank,perc_wet_stdev,overall_imp,perc_snow_severe_imp,perc_poor_viz_imp,perc_snow_imp,adjacc_imp,perc_dry_imp,perc_wet_imp,avgmetrics
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488,0.909275,1.000000,0.048005,0.750000,0.091588,0.722222,0.129333,0.888889,0.159973,1.000000,0.018741,0.944444,0.050844,1.000000,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.689180
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910,0.898581,0.916667,0.048005,0.861111,0.091588,0.805556,0.129333,1.000000,0.159973,0.888889,0.018741,0.861111,0.050844,0.888889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.686288
22,107445,72289,7635,5783,14945,9176,42290,30099,38000,23972,4575,3259,96364,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:33:33.641890,0.672800,0.630842,0.712350,0.613985,0.757433,0.711729,0.685268,0.896868,0.861111,0.048005,0.944444,0.091588,0.777778,0.129333,0.972222,0.159973,0.833333,0.018741,0.777778,0.050844,0.833333,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.682151
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796,0.906482,0.944444,0.048005,0.777778,0.091588,0.694444,0.129333,0.833333,0.159973,0.944444,0.018741,0.972222,0.050844,0.861111,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.680738
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098,0.909107,0.972222,0.048005,0.708333,0.091588,0.666667,0.129333,0.750000,0.159973,0.972222,0.018741,0.916667,0.050844,0.972222,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.679381
28,107445,71027,7635,5791,14945,9061,42290,28954,38000,23838,4575,3383,95382,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:15:03.281649,0.661054,0.627316,0.739454,0.606290,0.758481,0.684654,0.683239,0.887729,0.805556,0.048005,0.972222,0.091588,0.916667,0.129333,0.916667,0.159973,0.694444,0.018741,0.750000,0.050844,0.638889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.677693
8,107445,72760,7635,5625,14945,8915,42290,30671,38000,24421,4575,3128,97003,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 03:55:06.972875,0.677184,0.642658,0.683716,0.596521,0.736739,0.725254,0.676977,0.902815,0.888889,0.048005,0.708333,0.091588,0.638889,0.129333,0.861111,0.159973,0.916667,0.018741,0.888889,0.050844,0.944444,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.677029
10,107445,71241,7635,5837,14945,9087,42290,29057,38000,24004,4575,3256,95638,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 04:08:26.195298,0.663046,0.631684,0.711694,0.608029,0.764506,0.687089,0.680600,0.890111,0.833333,0.048005,1.000000,0.091588,0.750000,0.129333,0.944444,0.159973,0.722222,0.018741,0.805556,0.050844,0.666667,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.676212
23,107445,70746,7635,5779,14945,8490,42290,2

IMPORTANCES


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,adjacc,overall_PercentileRank,overall_stdev,perc_snow_severe_PercentileRank,perc_snow_severe_stdev,perc_poor_viz_PercentileRank,perc_poor_viz_stdev,perc_snow_PercentileRank,perc_snow_stdev,adjacc_PercentileRank,adjacc_stdev,perc_dry_PercentileRank,perc_dry_stdev,perc_wet_PercentileRank,perc_wet_stdev,overall_imp,perc_snow_severe_imp,perc_poor_viz_imp,perc_snow_imp,adjacc_imp,perc_dry_imp,perc_wet_imp,avgmetrics,weighted_avg_recall
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910,0.898581,0.916667,0.048005,0.861111,0.091588,0.805556,0.129333,1.000000,0.159973,0.888889,0.018741,0.861111,0.050844,0.888889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.686288,0.695069
28,107445,71027,7635,5791,14945,9061,42290,28954,38000,23838,4575,3383,95382,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:15:03.281649,0.661054,0.627316,0.739454,0.606290,0.758481,0.684654,0.683239,0.887729,0.805556,0.048005,0.972222,0.091588,0.916667,0.129333,0.916667,0.159973,0.694444,0.018741,0.750000,0.050844,0.638889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.677693,0.692507
22,107445,72289,7635,5783,14945,9176,42290,30099,38000,23972,4575,3259,96364,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:33:33.641890,0.672800,0.630842,0.712350,0.613985,0.757433,0.711729,0.685268,0.896868,0.861111,0.048005,0.944444,0.091588,0.777778,0.129333,0.972222,0.159973,0.833333,0.018741,0.777778,0.050844,0.833333,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.682151,0.692420
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488,0.909275,1.000000,0.048005,0.750000,0.091588,0.722222,0.129333,0.888889,0.159973,1.000000,0.018741,0.944444,0.050844,1.000000,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.689180,0.690397
10,107445,71241,7635,5837,14945,9087,42290,29057,38000,24004,4575,3256,95638,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 04:08:26.195298,0.663046,0.631684,0.711694,0.608029,0.764506,0.687089,0.680600,0.890111,0.833333,0.048005,1.000000,0.091588,0.750000,0.129333,0.944444,0.159973,0.722222,0.018741,0.805556,0.050844,0.666667,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.676212,0.689624
23,107445,70746,7635,5779,14945,8490,42290,29357,38000,23759,4575,3361,96445,nestcv_5cat_twotrain,_A_densenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:43:05.127303,0.658439,0.625237,0.734645,0.568083,0.756909,0.694183,0.675811,0.897622,0.777778,0.048005,0.916667,0.091588,0.861111,0.129333,0.666667,0.159973,0.861111,0.018741,0.694444,0.050844,0.722222,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.671468,0.683904
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796,0.906482,0.944444,0.048005,0.777778,0.091588,0.694444,0.129333,0.833333,0.159973,0.944444,0.018741,0.972222,0.050844,0.861111,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.680738,0.683839
8,107445,72760,7635,5625,14945,8915,42290,30671,38000,24421,4575,3128,97003,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 03:55:06.972875,0.677184,0.642658,0.683716,0.596521,0.736739,0.725254,0.676977,0.902815,0.888889,0.048005,0.708333,0.091588,0.638889,0.129333,0.861111,0.159973,0.916667,0.018741,0.888889,0.050844,0.944444,0.033

IMPORTANCE + AVG


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,adjacc,overall_PercentileRank,overall_stdev,perc_snow_severe_PercentileRank,perc_snow_severe_stdev,perc_poor_viz_PercentileRank,perc_poor_viz_stdev,perc_snow_PercentileRank,perc_snow_stdev,adjacc_PercentileRank,adjacc_stdev,perc_dry_PercentileRank,perc_dry_stdev,perc_wet_PercentileRank,perc_wet_stdev,overall_imp,perc_snow_severe_imp,perc_poor_viz_imp,perc_snow_imp,adjacc_imp,perc_dry_imp,perc_wet_imp,avgmetrics,weighted_avg_recall,impAndAvg
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488,0.909275,1.000000,0.048005,0.750000,0.091588,0.722222,0.129333,0.888889,0.159973,1.000000,0.018741,0.944444,0.050844,1.000000,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.689180,0.690397,0.690047
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910,0.898581,0.916667,0.048005,0.861111,0.091588,0.805556,0.129333,1.000000,0.159973,0.888889,0.018741,0.861111,0.050844,0.888889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.686288,0.695069,0.687467
22,107445,72289,7635,5783,14945,9176,42290,30099,38000,23972,4575,3259,96364,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:33:33.641890,0.672800,0.630842,0.712350,0.613985,0.757433,0.711729,0.685268,0.896868,0.861111,0.048005,0.944444,0.091588,0.777778,0.129333,0.972222,0.159973,0.833333,0.018741,0.777778,0.050844,0.833333,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.682151,0.692420,0.683496
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796,0.906482,0.944444,0.048005,0.777778,0.091588,0.694444,0.129333,0.833333,0.159973,0.944444,0.018741,0.972222,0.050844,0.861111,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.680738,0.683839,0.681733
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098,0.909107,0.972222,0.048005,0.708333,0.091588,0.666667,0.129333,0.750000,0.159973,0.972222,0.018741,0.916667,0.050844,0.972222,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.679381,0.679692,0.680341
28,107445,71027,7635,5791,14945,9061,42290,28954,38000,23838,4575,3383,95382,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:15:03.281649,0.661054,0.627316,0.739454,0.606290,0.758481,0.684654,0.683239,0.887729,0.805556,0.048005,0.972222,0.091588,0.916667,0.129333,0.916667,0.159973,0.694444,0.018741,0.750000,0.050844,0.638889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.677693,0.692507,0.678933
8,107445,72760,7635,5625,14945,8915,42290,30671,38000,24421,4575,3128,97003,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 03:55:06.972875,0.677184,0.642658,0.683716,0.596521,0.736739,0.725254,0.676977,0.902815,0.888889,0.048005,0.708333,0.091588,0.638889,0.129333,0.861111,0.159973,0.916667,0.018741,0.888889,0.050844,0.944444,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.677029,0.680249,0.678137
10,107445,71241,7635,5837,14945,9087,42290,29057,38000,24004,4575,3256,95638,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 04:08:26.195298,0.663046,0.631684,0.711694,0.608029,0.764506,0.687089,0.680600,0.890111,0.833333,0.048005,1.000000,0.091588,0.750000,0.1293

IMPORTANCES OLD PAPER


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,adjacc,overall_PercentileRank,overall_stdev,perc_snow_severe_PercentileRank,perc_snow_severe_stdev,perc_poor_viz_PercentileRank,perc_poor_viz_stdev,perc_snow_PercentileRank,perc_snow_stdev,adjacc_PercentileRank,adjacc_stdev,perc_dry_PercentileRank,perc_dry_stdev,perc_wet_PercentileRank,perc_wet_stdev,overall_imp,perc_snow_severe_imp,perc_poor_viz_imp,perc_snow_imp,adjacc_imp,perc_dry_imp,perc_wet_imp,avgmetrics,weighted_avg_recall,impAndAvg,importanceold
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910,0.898581,0.916667,0.048005,0.861111,0.091588,0.805556,0.129333,1.000000,0.159973,0.888889,0.018741,0.861111,0.050844,0.888889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.686288,0.695069,0.687467,0.699483
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488,0.909275,1.000000,0.048005,0.750000,0.091588,0.722222,0.129333,0.888889,0.159973,1.000000,0.018741,0.944444,0.050844,1.000000,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.689180,0.690397,0.690047,0.698859
22,107445,72289,7635,5783,14945,9176,42290,30099,38000,23972,4575,3259,96364,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:33:33.641890,0.672800,0.630842,0.712350,0.613985,0.757433,0.711729,0.685268,0.896868,0.861111,0.048005,0.944444,0.091588,0.777778,0.129333,0.972222,0.159973,0.833333,0.018741,0.777778,0.050844,0.833333,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.682151,0.692420,0.683496,0.696242
28,107445,71027,7635,5791,14945,9061,42290,28954,38000,23838,4575,3383,95382,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:15:03.281649,0.661054,0.627316,0.739454,0.606290,0.758481,0.684654,0.683239,0.887729,0.805556,0.048005,0.972222,0.091588,0.916667,0.129333,0.916667,0.159973,0.694444,0.018741,0.750000,0.050844,0.638889,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.677693,0.692507,0.678933,0.693679
10,107445,71241,7635,5837,14945,9087,42290,29057,38000,24004,4575,3256,95638,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 04:08:26.195298,0.663046,0.631684,0.711694,0.608029,0.764506,0.687089,0.680600,0.890111,0.833333,0.048005,1.000000,0.091588,0.750000,0.129333,0.944444,0.159973,0.722222,0.018741,0.805556,0.050844,0.666667,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.676212,0.689624,0.677757,0.691821
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796,0.906482,0.944444,0.048005,0.777778,0.091588,0.694444,0.129333,0.833333,0.159973,0.944444,0.018741,0.972222,0.050844,0.861111,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.680738,0.683839,0.681733,0.691566
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098,0.909107,0.972222,0.048005,0.708333,0.091588,0.666667,0.129333,0.750000,0.159973,0.972222,0.018741,0.916667,0.050844,0.972222,0.033426,1.0,1.0,0.9,0.9,0.2,0.8,0.8,0.679381,0.679692,0.680341,0.688966
8,107445,72760,7635,5625,14945,8915,42290,30671,38000,24421,4575,3128,97003,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugFalse,2025-06-29 03:55:06.972875,0.677184,0.642658,0.683716,0.596521,0.736739,0

### CNN - Hyperparameter tuning (HT) selection

In [13]:
##### for HT model selection

# for each of the 4 BL models that hyp tuning was performed, grab the best model from each, and move forward with HTing EACH of those for the + weather data. 

df = pd.read_csv("/home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv")

df["overall"] = df["correctims"]/df["totalims"]

for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)


# add col with the BL model type (i.e., the 4 BL models) from which we are then selecting the best one hyperparams
model_BL_desc = []
for i in df["exp_details"]:
    b = i.find("L2")
    e = i.rfind("Aug")
    desc = i[:b]+i[e:]
    model_BL_desc.append(desc)

df["model_BL_desc"] = model_BL_desc

unique_BL = np.unique(df["model_BL_desc"])


### The way I originally did it with selecting top one model PER each baseline model
for bl in unique_BL:
    dfsub = df[df["model_BL_desc"]==bl]
    if len(dfsub)>1: # only look at the selected 4 BL models, not all the other 32 with only the one BL run exp
        print(len(dfsub))
        top4_avgrecall = dfsub.sort_values("avg_recall", ascending=False)[0:5] # since only 16 HT are run, select from top 5 (roughly 1/3)
        selected1 = top4_avgrecall.sort_values("overall", ascending=False)[0:5] # grab the top 1
        print(bl)
        display(selected1)
        print(selected1["exp_details"])

17
_A_mobilenet_TRLETrue_ASTFalse_AugFalse


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
57,107445,71893,7635,5911,14945,9531,42290,29259,38000,23703,4575,3489,95897,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,2025-07-08 18:38:11.556300,0.669114,0.623763,0.762623,0.637738,0.774198,0.691866,0.698038,_A_mobilenet_TRLETrue_ASTFalse_AugFalse
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse
63,107445,70629,7635,5967,14945,9774,42290,28085,38000,22988,4575,3815,94838,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse,2025-07-08 18:58:01.654365,0.657350,0.604947,0.833880,0.653998,0.781532,0.664105,0.707693,_A_mobilenet_TRLETrue_ASTFalse_AugFalse
67,107445,67831,7635,5966,14945,9949,42290,26134,38000,21895,4575,3887,94130,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugFalse,2025-07-08 19:11:14.396091,0.631309,0.576184,0.849617,0.665708,0.781401,0.617971,0.698176,_A_mobilenet_TRLETrue_ASTFalse_AugFalse


57    _A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse
55      _A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse
59      _A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
63      _A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse
67      _A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugFalse
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTFalse_AugFalse


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse
92,107445,72278,7635,5809,14945,9048,42290,29645,38000,24258,4575,3518,97087,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_6_E75_AugFalse,2025-07-14 12:27:30.975803,0.672698,0.638368,0.768962,0.605420,0.760838,0.700993,0.694916,_A_resnet_TRLETrue_ASTFalse_AugFalse
91,107445,72217,7635,5852,14945,9077,42290,29795,38000,23749,4575,3744,96513,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-14 03:26:19.049010,0.672130,0.624974,0.818361,0.607360,0.766470,0.704540,0.704341,_A_resnet_TRLETrue_ASTFalse_AugFalse
93,107445,71778,7635,5873,14945,9131,42290,29136,38000,23861,4575,3777,96481,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse,2025-07-14 12:34:44.288337,0.668044,0.627921,0.825574,0.610974,0.769221,0.688957,0.704529,_A_resnet_TRLETrue_ASTFalse_AugFalse
97,107445,69817,7635,5912,14945,9250,42290,27322,38000,23486,4575,3847,95827,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugFalse,2025-07-14 13:04:10.627751,0.649793,0.618053,0.840874,0.618936,0.774329,0.646063,0.699651,_A_resnet_TRLETrue_ASTFalse_AugFalse


87      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse
92    _A_resnet_TRLETrue_ASTFalse_L20_001_DR0_6_E75_AugFalse
91      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
93      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse
97      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugFalse
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTFalse_AugTrue


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue
83,107445,69799,7635,5900,14945,9260,42290,26803,38000,23934,4575,3902,95796,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugTrue,2025-07-14 02:27:28.843482,0.649625,0.629842,0.852896,0.619605,0.772757,0.633790,0.701778,_A_resnet_TRLETrue_ASTFalse_AugTrue


74    _A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue
71      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue
75      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
79      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue
83      _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugTrue
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTTrue_AugTrue


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue
44,107445,73267,7635,5855,14945,9250,42290,29971,38000,24789,4575,3402,97270,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L21e-07_DR0_6_E75_AugTrue,2025-07-08 17:40:51.120456,0.681902,0.652342,0.743607,0.618936,0.766863,0.708702,0.698090,_A_resnet_TRLETrue_ASTTrue_AugTrue


39      _A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
47      _A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue
43      _A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue
51      _A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue
44    _A_resnet_TRLETrue_ASTTrue_L21e-07_DR0_6_E75_AugTrue
Name: exp_details, dtype: object


In [12]:
# TESTING 7/26 -  1) what percentile does it fall in for each. 2) what is the spread within that percentile (e.g. if all super similar, then the performance of that class doesnt matter as much, so weight its placement accordingly (i.e. if all models get between 70-71% on dry, then what does the top 99% model matter when it's just spread of 1%? 3) what do I care about them? Our model importances

df["adjacc"] = df["ok"]/df["totalims"]



# overall_imp = 1.0
perc_snow_severe_imp = 1.0
perc_poor_viz_imp = 1.0
perc_snow_imp = 1.0
adjacc_imp = 0.1
perc_dry_imp = 1.0
perc_wet_imp = 1.0

# overall_imp = 1.0
# perc_snow_severe_imp = 1.0
# perc_poor_viz_imp = 0.9
# perc_snow_imp = 0.9
# adjacc_imp = 0.2
# perc_dry_imp = 0.8
# perc_wet_imp = 0.8

overall_std = df["overall"].std(axis=0)
perc_snow_severe_std = df["perc_snow_severe"].std(axis=0)
perc_poor_viz_std = df["perc_poor_viz"].std(axis=0)
perc_snow_std = df["perc_snow"].std(axis=0)
adjacc_std = df["adjacc"].std(axis=0)
perc_dry_std = df["perc_dry"].std(axis=0)
perc_wet_std = df["perc_wet"].std(axis=0)

std_list = [overall_std,perc_snow_severe_std,perc_poor_viz_std,perc_snow_std,adjacc_std,perc_dry_std,perc_wet_std]
# stdev weights alone are too big, want them to only range in scale between 0.8 and 1.0, o/w it will overpower the assigned weights. Rescale them

def rescale_list(data_list, new_min=0.8, new_max=1.0):
    # Handle empty list case
    if not data_list:
        return []

    # Calculate the minimum and maximum values of the original list
    original_min = min(data_list)
    original_max = max(data_list)

    # Handle the case where all values are the same to avoid division by zero
    if original_min == original_max:
        return [new_min] * len(data_list)

    # Apply the min-max scaling formula to each value
    rescaled_list = [
        ((x - original_min) / (original_max - original_min)) * (new_max - new_min) + new_min
        for x in data_list
    ]

    return rescaled_list

std_scaled = rescale_list(std_list, new_min=0.8, new_max=1.0)

overall_wt = overall_imp*std_scaled[0]
overperc_snow_severe_wt = perc_snow_severe_imp*std_scaled[1]
perc_poor_viz_wt = perc_poor_viz_imp*std_scaled[2]
perc_snow_wt = perc_snow_imp*std_scaled[3]
adjacc_wt = adjacc_imp*std_scaled[4]
dry_wt = perc_dry_imp*std_scaled[5]
perc_wet_wt = perc_wet_imp*std_scaled[6]

importances_new = overall_wt,overperc_snow_severe_wt,perc_poor_viz_wt,perc_snow_wt,adjacc_wt,dry_wt,perc_wet_wt

scale = sum(importances_new)

print(importances_new)

print(scale)

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importances_new"] = (values * importances_new).sum(axis=1) / scale
importances_df= df.sort_values("importances_new", ascending=False)[0:50]
display(importances_df)


sub = df[((df["model_BL_desc"].str.contains
("_A_resnet_TRLETrue_ASTFalse_AugTrue"))|(df["exp_desc"].str.contains("_A_mobilenet_TRLETrue_ASTFalse_AugFalse")))]
importances_sub = sub.sort_values("importances_new", ascending=False) 
# _A_mobilenet_TRLETrue_ASTFalse_AugFalse
# _A_resnet_TRLETrue_ASTFalse_AugTrue

print(len(sub))
display(importances_sub[0:20])

# df['Row_STDEV'] = df[['A', 'B', 'C']].std(axis=1)

(0.8458682479405978, 0.9078354459634085, 0.9740836971541394, 1.0, 0.08000000000000002, 0.843078554245977, 0.843079396487979)
5.493945341792101


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,adjacc,importances_new
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.904537,0.710179
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.901959,0.709651
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.907292,0.709339
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.898944,0.707857
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.905198,0.707846
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.899800,0.707561
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.885104,0.706466
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.886677,0.706405
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.900330,0.705352
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.897520,0.704965


17


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,adjacc,importances_new
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.898944,0.707857
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.899800,0.707561
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.897520,0.704965
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.906185,0.701775
76,107445,72244,7635,5835,14945,9136,42290,28715,38000,24953,4575,3605,96759,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-07_DR0_6_E75_AugTrue,2025-07-14 01:36:06.091513,0.672381,0.656658,0.787978,0.611308,0.764244,0.679002,0.699838,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.900544,0.698865
72,107445,73484,7635,5824,14945,9091,42290,30280,38000,24853,4575,3436,97394,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-07_DR0_4_E75_AugTrue,2025-07-14 01:07:01.594115,0.683922,0.654026,0.751038,0.608297,0.762803,0.716009,0.698435,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.906454,0.698668
83,107445,69799,7635,5900,14945,9260,42290,26803,38000,23934,4575,3902,95796,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugTrue,2025-07-14 02:27:28.843482,0.649625,0.629842,0.852896,0.619605,0.772757,0.633790,0.701778,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.891582,0.698605
78,107445,72508,7635,5902,14945,9103,42290,29239,38000,24770,4575,3494,96947,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_6_E75_AugTrue,2025-07-14 01:51:02.250187,0.674838,0.651842,0.763716,0.609100,0.773019,0.691393,0.697814,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.902294,0.697178
73,107445,73391,7635,5778,14945,9185,42290,30298,38000,24729,4575,3401,97305,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugTrue,2025-07-14 01:14:15.499454,0.683056,0.650763,0.743388,0.614587,0.756778,0.716434,0.696390,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.905626,0.696880
77,107445,72541,7635,5842,14945,9174,42290,29455,38000,24586,4575,3484,96806,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-05_DR0_6_E75_AugTrue,2025-07-14 01:43:36.817135,0.675145,0.647000,0.761530,0.613851,0.765160,0.696500,0.696808,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.900982,0.696426


In [27]:
# TESTING SOME OTHER VARIATIONS


### Trying where I select just the top ONE
top4_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10] # since only 16 HT are run, select from top 5 (roughly 1/3)
selected1 = top4_avgrecall.sort_values("overall", ascending=False)[0:1] 
display(selected1)


# TESTING SOME OTHER VARIATIONS
print("AVERAGE WAY")
for bl in unique_BL:
    dfsub = df[df["model_BL_desc"]==bl]
    if len(dfsub)>1: # only look at the selected 4 BL models, not all the other 32 with only the one BL run exp
        print(len(dfsub))
        # dfsub["avgmetrics"] = dfsub[['overall', 'avg_recall']].mean(axis=1)
        dfsub["avgmetrics"] = 0.5*dfsub['overall']+0.5*dfsub['avg_recall']
        avgmetrics = dfsub.sort_values("avgmetrics", ascending=False)[0:1]
        display(avgmetrics)# grab the top 1

print("AVG OVERALL (not PER BL model)")
df["avgmetrics"] = 0.25*df['overall']+0.75*df['avg_recall']
avgmetrics = df.sort_values("avgmetrics", ascending=False)[0:10]
display(avgmetrics)

print("IMPORTANCES")
importances = [2,1.5,1,1,1.5]
cols = ["perc_snow_severe", "perc_snow", "perc_wet", "perc_dry", "perc_poor_viz"]
values = df[cols].values
df["weighted_avg_recall"] = (values * importances).sum(axis=1) / sum(importances)
wtdavg = df.sort_values("weighted_avg_recall", ascending=False)[0:30]
display(wtdavg)

print("IMPORTANCE + AVG")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["impAndAvg"] = (1/3)*df['overall']+(1/3)*df['avg_recall']+(1/3)*df["weighted_avg_recall"] 
impAndAvg = df.sort_values("impAndAvg", ascending=False)[0:10]
display(impAndAvg)

print("IMPORTANCES OLD PAPER")
importancesOld = [10, 10, 8, 8, 2, 5, 5]
df["adjacc"] = df["ok"]/df["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)
importanceold = df.sort_values("importanceold", ascending=False)[0:10]
display(importanceold)
print("note! If theoretically we only ran the top 2 BLs from BL, then we should filter to only those top 2 for making the selection here... ")

print("redoing run with only the top 2 from BL...")
print("grab one BL")
# BL1_strings = ['A_resnet_TRLETrue_ASTFalse','AugTrue']
# BL2_strings = ['A_mobilenet_TRLETrue_ASTFalse', 'AugFalse']
# # Construct the regular expression pattern
# BL1_pattern = '|'.join(BL1_strings)
# BL2_pattern = '|'.join(BL2_strings)
# # Filter the DataFrame
# print(len(df))
# exarch = df[(df['exp_details'].str.contains(BL1_pattern, na=False))|(df['exp_details'].str.contains(BL2_pattern, na=False))] # na=False handles potential NaN values
# print(len(exarch))

# Individual conditions
resnet_cond = df['exp_details'].str.contains('A_resnet_TRLETrue_ASTFalse', na=False) & df['exp_details'].str.contains('AugTrue', na=False)

mobilenet_cond = df['exp_details'].str.contains('A_mobilenet_TRLETrue_ASTFalse', na=False) & df['exp_details'].str.contains('AugFalse', na=False)
print(len(df))
# Combine the conditions
exarch = df[resnet_cond | mobilenet_cond]
print(len(exarch))
importanceold_BL2 = exarch.sort_values("importanceold", ascending=False)
display(importanceold_BL2)


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue


AVERAGE WAY
17


/tmp/tmp.80hJAYorXg/ipykernel_4194109/1610897224.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsub["avgmetrics"] = 0.5*dfsub['overall']+0.5*dfsub['avg_recall']


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.64269,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.687617


17


...

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.719289,0.705972
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.716765,0.705945
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.721038,0.704769
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.715769,0.703915
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.720697,0.701918
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.718010,0.701799
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.716073,0.699977
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.722183,0.699139
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.722849,0.698593
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697230,0.708469,0.697970


IMPORTANCES OLD PAPER


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.721038,0.704769,0.901959,0.720173
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.719289,0.705972,0.904537,0.720064
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.716765,0.705945,0.907292,0.718928
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.720697,0.701918,0.898944,0.718561
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.722849,0.698593,0.885104,0.717695
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.722183,0.699139,0.886677,0.717666
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.715769,0.703915,0.905198,0.717416
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.718010,0.701799,0.899800,0.717285
63,107445,70629,7635,5967,14945,9774,42290,28085,38000,22988,4575,3815,94838,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse,2025-07-08 18:58:01.654365,0.657350,0.604947,0.833880,0.653998,0.781532,0.664105,0.707693,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.695107,0.723419,0.696154,0.882666,0.716718
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.716073,0.699977,0.900330,0.715544


note! If theoretically we only ran the top 2 BLs from BL, then we should filter to only those top 2 for making the selection here... 
redoing run with only the top 2 from BL...
grab one BL
100
34


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.720697,0.701918,0.898944,0.718561
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.722849,0.698593,0.885104,0.717695
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.722183,0.699139,0.886677,0.717666
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.718010,0.701799,0.899800,0.717285
63,107445,70629,7635,5967,14945,9774,42290,28085,38000,22988,4575,3815,94838,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse,2025-07-08 18:58:01.654365,0.657350,0.604947,0.833880,0.653998,0.781532,0.664105,0.707693,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.695107,0.723419,0.696154,0.882666,0.716718
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697516,0.717923,0.697963,0.897520,0.715313
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697230,0.708469,0.697970,0.906185,0.711217
83,107445,69799,7635,5900,14945,9260,42290,26803,38000,23934,4575,3902,95796,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_8_E75_AugTrue,2025-07-14 02:27:28.843482,0.649625,0.629842,0.852896,0.619605,0.772757,0.633790,0.701778,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.688740,0.716843,0.689415,0.891582,0.710524
76,107445,72244,7635,5835,14945,9136,42290,28715,38000,24953,4575,3605,96759,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-07_DR0_6_E75_AugTrue,2025-07-14 01:36:06.091513,0.672381,0.656658,0.787978,0.611308,0.764244,0.679002,0.699838,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.692974,0.709011,0.693743,0.900544,0.709165
72,107445,73484,7635,5824,14945,9091,42290,30280,38000,24853,4575,3436,97394,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-07_DR0_4_E75_AugTrue,2025-07-14 01:07:01.594115,0.683922,0.654026,0.751038,0.608297,0.762803,0.716009,0.698435,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.694806,0.704949,0.695769,0.906454,0.708438


In [30]:
print("IMPORTANCES OLD PAPER VARIATION 7/26")
importancesOld = [10, 10, 9, 9, 2, 7, 8]
df["adjacc"] = df["ok"]/df["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)
importanceold = df.sort_values("importanceold", ascending=False)[0:10]
display(importanceold)
print("note! If theoretically we only ran the top 2 BLs from BL, then we should filter to only those top 2 for making the selection here... ")

print("redoing run with only the top 2 from BL...")
print("grab one BL")

IMPORTANCES OLD PAPER VARIATION 7/26


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.719289,0.705972,0.904537,0.716831
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.721038,0.704769,0.901959,0.716418
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.716765,0.705945,0.907292,0.716162
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.715769,0.703915,0.905198,0.714490
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.720697,0.701918,0.898944,0.714477
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.718010,0.701799,0.899800,0.713722
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.722183,0.699139,0.886677,0.712672
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.722849,0.698593,0.885104,0.712450
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.716073,0.699977,0.900330,0.711997
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697516,0.717923,0.697963,0.897520,0.710877


note! If theoretically we only ran the top 2 BLs from BL, then we should filter to only those top 2 for making the selection here... 
redoing run with only the top 2 from BL...
grab one BL


In [36]:
5/17

0.29411764705882354

In [39]:
10/36

0.2777777777777778

In [40]:
54960*.28

15388.800000000001

# Downstream model
- After models are run (see dir: /home/csutter/DRIVE-clean/downstream), results are appended to here: /home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv
- First chunk below aggregates results by model since the runs were appended by fold, which we need to aggregate across.
- Then can select, similar to baseline/HT selection of CNN, to select best from that aggregated file -- last chunk of code

In [2]:
# Already ran this aggregation code, don't need to run again!

# Important first step: That csv has each model|fold innerVal results appended, but need to aggregate across the 30 folds to make selection of best model, so FIRST need to preprocess to aggregate at the model level, rather than model|fold level

results_model_fold = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv")

results_model_fold.head(4)

aggregated = results_model_fold.groupby(["exp_desc","exp_details"]).sum().reset_index()
aggregated.head(5)

# Save this out so don't need to run again
aggregated.to_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

/tmp/tmp.80hJAYorXg/ipykernel_4194109/4133674865.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated = results_model_fold.groupby(["exp_desc","exp_details"]).sum().reset_index()


In [3]:
results_model_fold = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv")

results_model_fold.head(4)

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,datafold
0,3697,2814,294,178,511,297,1422,1154,1326,1086,144,99,3451,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 0.01}",nestcv_5cat_twotrain_OT1_m3_T5V0
1,3697,2800,294,177,511,304,1422,1140,1326,1086,144,93,3448,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 0.1}",nestcv_5cat_twotrain_OT1_m3_T5V0
2,3697,2789,294,175,511,308,1422,1122,1326,1100,144,84,3431,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 1.0}",nestcv_5cat_twotrain_OT1_m3_T5V0
3,3697,2791,294,177,511,307,1422,1123,1326,1101,144,83,3429,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 10.0}",nestcv_5cat_twotrain_OT1_m3_T5V0


In [4]:
print(np.unique(results_model_fold["exp_desc"]))

['A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse'
 'A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse'
 'A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue'
 'A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse'
 'A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue'
 'A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue']


### Downstream model selection

In [16]:
df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]



df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

# grab one from  a specific arch
archs = []
for i in range(0,len(df)):
    desc = df["exp_desc"][i]
    desc = desc[2:]
    end = desc.find("_")
    a = desc[:end]
    archs.append(a)

df["arch"] = archs

print(len(df))


# Decision: index 1507 showed up in a variety of different weighted-importances (when we tried these below), and replaced the initial top 10 with weighted recall rather than just plain average. Motivation to keep it simple and just move forward with the simplest -- top 10 avg recall, then take the max overall from that. Top models are all pretty similar, anyway. 
top10_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:1]

display(selected4)

print(selected4["exp_details"])
print(np.max(selected4["perc_snow"]))

# Look at that from each arch.. maybe justify taking the mobilenet one??
# Decision: index 1507 showed up in a variety of different weighted-importances (when we tried these below), and replaced the initial top 10 with weighted recall rather than just plain average. Motivation to keep it simple and just move forward with the simplest -- top 10 avg recall, then take the max overall from that. Top models are all pretty similar, anyway. 
resttop = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:1]

display(selected4)

print(selected4["exp_details"])
print(np.max(selected4["perc_snow"]))

2742


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch
2421,2421,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet


2421    rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
Name: exp_details, dtype: object
0.748745399799264


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch
2421,2421,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet


2421    rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
Name: exp_details, dtype: object
0.748745399799264


In [6]:
len(df)

2742

In [28]:
# TESTING 7/26 -  1) what percentile does it fall in for each. 2) what is the spread within that percentile (e.g. if all super similar, then the performance of that class doesnt matter as much, so weight its placement accordingly (i.e. if all models get between 70-71% on dry, then what does the top 99% model matter when it's just spread of 1%? 3) what do I care about them? Our model importances

df["adjacc"] = df["ok"]/df["totalims"]



overall_imp = 1.0
perc_snow_severe_imp = 1.0
perc_poor_viz_imp = 0.9#0.9
perc_snow_imp = 0.9
adjacc_imp = 0.2
perc_dry_imp = 0.8
perc_wet_imp = 0.8

overall_std = df["overall"].std(axis=0)
perc_snow_severe_std = df["perc_snow_severe"].std(axis=0)
perc_poor_viz_std = df["perc_poor_viz"].std(axis=0)
perc_snow_std = df["perc_snow"].std(axis=0)
adjacc_std = df["adjacc"].std(axis=0)
perc_dry_std = df["perc_dry"].std(axis=0)
perc_wet_std = df["perc_wet"].std(axis=0)

std_list = [overall_std,perc_snow_severe_std,perc_poor_viz_std,perc_snow_std,adjacc_std,perc_dry_std,perc_wet_std]
# stdev weights alone are too big, want them to only range in scale between 0.8 and 1.0, o/w it will overpower the assigned weights. Rescale them

def rescale_list(data_list, new_min=0.8, new_max=1.0):
    # Handle empty list case
    if not data_list:
        return []

    # Calculate the minimum and maximum values of the original list
    original_min = min(data_list)
    original_max = max(data_list)

    # Handle the case where all values are the same to avoid division by zero
    if original_min == original_max:
        return [new_min] * len(data_list)

    # Apply the min-max scaling formula to each value
    rescaled_list = [
        ((x - original_min) / (original_max - original_min)) * (new_max - new_min) + new_min
        for x in data_list
    ]

    return rescaled_list

std_scaled = rescale_list(std_list, new_min=0.8, new_max=1.0)

overall_wt = overall_imp*std_scaled[0]
overperc_snow_severe_wt = perc_snow_severe_imp*std_scaled[1]
perc_poor_viz_wt = perc_poor_viz_imp*std_scaled[2]
perc_snow_wt = perc_snow_imp*std_scaled[3]
adjacc_wt = adjacc_imp*std_scaled[4]
dry_wt = perc_dry_imp*std_scaled[5]
perc_wet_wt = perc_wet_imp*std_scaled[6]

importances_new = overall_wt,overperc_snow_severe_wt,perc_poor_viz_wt,perc_snow_wt,adjacc_wt,dry_wt,perc_wet_wt

scale = sum(importances_new)

print(importances_new)

print(scale)

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importances_new"] = (values * importances_new).sum(axis=1) / scale
importances_df= df.sort_values("importances_new", ascending=False)[0:10]
# display(importances_df)

# Assuming we only run the selected i=20 and 4, then the best here would be i = 1988	A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue	rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
#7/26 -- TRY RUNNING THIS AS ALTERNATE2

sub = df[((df["exp_desc"].str.contains
("A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue"))|(df["exp_desc"].str.contains("A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse")))]
importances_sub = sub.sort_values("importances_new", ascending=False) #A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse

print(len(sub))
display(importances_sub[0:20])



(0.9325876481510482, 0.8900147249715423, 0.8615608871261611, 0.8640759472640364, 0.16000000000000003, 0.8, 0.7610198801269549)
5.2692590876397425
457


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,adjacc,importances_new
136,136,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86353,7635,5871,14945,11170,42290,32605,38000,33510,4575,3197,101430,0.803695,0.881842,0.698798,0.747407,0.768959,0.770986,0.773598,mobilenet,0.944018,0.782847
154,154,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86374,7635,5862,14945,11178,42290,32586,38000,33578,4575,3170,101455,0.803890,0.883632,0.692896,0.747942,0.767780,0.770537,0.772557,mobilenet,0.944251,0.782020
208,208,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86604,7635,5868,14945,11066,42290,33061,38000,33452,4575,3157,101561,0.806031,0.880316,0.690055,0.740448,0.768566,0.781769,0.772231,mobilenet,0.945237,0.781986
172,172,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86383,7635,5864,14945,11191,42290,32593,38000,33578,4575,3157,101482,0.803974,0.883632,0.690055,0.748812,0.768042,0.770702,0.772249,mobilenet,0.944502,0.781788
142,142,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86334,7635,5886,14945,11173,42290,32583,38000,33538,4575,3154,101452,0.803518,0.882579,0.689399,0.747608,0.770923,0.770466,0.772195,mobilenet,0.944223,0.781687
124,124,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86248,7635,5886,14945,11110,42290,32579,38000,33496,4575,3177,101376,0.802718,0.881474,0.694426,0.743392,0.770923,0.770371,0.772117,mobilenet,0.943515,0.781473
352,352,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86575,7635,5876,14945,11076,42290,33084,38000,33403,4575,3136,101531,0.805761,0.879026,0.685464,0.741117,0.769614,0.782313,0.771507,mobilenet,0.944958,0.781349
340,340,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86448,7635,5874,14945,11020,42290,33024,38000,33364,4575,3166,101515,0.804579,0.878000,0.692022,0.737370,0.769352,0.780894,0.771528,mobilenet,0.944809,0.781188
196,196,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86449,7635,5876,14945,11047,42290,32996,38000,33374,4575,3156,101527,0.804588,0.878263,0.689836,0.739177,0.769614,0.780232,0.771424,mobilenet,0.944921,0.781121
160,160,A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86361,7635,5862,14945,11166,42290,32565,38000,33628,4575,3140,101488,0.803769,0.884947,0.686339,0.747140,0.767780,0.770040,0.771249,mobilenet,0.944558,0.780932


In [32]:
# TESTING SOME OTHER VARIATIONS
importances = [0.9,1.2,1.2,1.5,1]
cols = ["perc_snow_severe", "perc_snow", "perc_wet", "perc_dry", "perc_poor_viz"]
values = df[cols].values
df["weighted_avg_recall"] = (values * importances).sum(axis=1) / sum(importances)

print("AVERAGE WAY")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["avgmetrics"] = 0.25*df['overall']+0.75*df['avg_recall']

avgmetrics = df.sort_values("avgmetrics", ascending=False)[0:10]
display(avgmetrics)

print("IMPORTANCES WAY")
top10_importance = df.sort_values("weighted_avg_recall", ascending=False)[0:10]
display(top10_importance)
# just for testing. Tried: 
# importances = [1,2,2,3,1.5]
# importances = [0.9,1.2,1.2,1.5,1]

print("IMPORTANCE + AVG")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["impAndAvg"] = (1/3)*df['overall']+(1/3)*df['avg_recall']+(1/3)*df["weighted_avg_recall"] 
impAndAvg = df.sort_values("impAndAvg", ascending=False)[0:20]
display(impAndAvg)


# grab example from specific arch
print("grab one arch specific")
exarch = df[df["arch"]=="mobilenet"].sort_values("avg_recall", ascending=False)[0:10]
# display(exarch)
top10_avgrecall = exarch.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:10]
display(selected4)

# grab one BL
print("grab one BL/HT specific")
exarch = df[df["exp_desc"]=="A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse"]
# display(exarch)
print(len(exarch))
top10_avgrecall = exarch.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:1]
display(selected4)
avgmetric = exarch.sort_values("avgmetrics", ascending=False)[0:10]
weighted = exarch.sort_values("weighted_avg_recall", ascending=False)[0:10]
display(avgmetric)
display(weighted)

print(len(df))

AVERAGE WAY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold,alg
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.702101,0.705972,0.904537,0.716831,ok
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.701636,0.705945,0.907292,0.716162,ok
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.700835,0.704769,0.901959,0.716418,ok
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.700039,0.703915,0.905198,0.714490,ok
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.699031,0.701799,0.899800,0.713722,ok
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.698324,0.701918,0.898944,0.714477,ok
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.696115,0.699977,0.900330,0.711997,ok
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.696043,0.699139,0.886677,0.712672,ok
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.696185,0.698593,0.885104,0.712450,ok
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697516,0.696333,0.697963,0.897520,0.710877,ok


IMPORTANCES WAY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold,alg
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.702101,0.705972,0.904537,0.716831,ok
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.701636,0.705945,0.907292,0.716162,ok
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.700835,0.704769,0.901959,0.716418,ok
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.700039,0.703915,0.905198,0.714490,ok
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.699031,0.701799,0.899800,0.713722,ok
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.698324,0.701918,0.898944,0.714477,ok
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697516,0.696333,0.697963,0.897520,0.710877,ok
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.696185,0.698593,0.885104,0.712450,ok
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.696115,0.699977,0.900330,0.711997,ok
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.696043,0.699139,0.886677,0.712672,ok


IMPORTANCE + AVG


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold,alg
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.701636,0.700902,0.907292,0.716162,ok
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.702101,0.700243,0.904537,0.716831,ok
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.700039,0.698671,0.905198,0.714490,ok
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.700835,0.698035,0.901959,0.716418,ok
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.699031,0.695472,0.899800,0.713722,ok
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.698324,0.694460,0.898944,0.714477,ok
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.696115,0.693324,0.900330,0.711997,ok
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697230,0.694357,0.693265,0.906185,0.708393,ok
72,107445,73484,7635,5824,14945,9091,42290,30280,38000,24853,4575,3436,97394,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L21e-07_DR0_4_E75_AugTrue,2025-07-14 01:07:01.594115,0.683922,0.654026,0.751038,0.608297,0.762803,0.716009,0.698435,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.694806,0.690994,0.691117,0.906454,0.705826,ok
79,107445,71872,7635,5854,14945,9068,42290,28468,38000,24638,4575,3844,96434,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugTrue,2025-07-14 01:58:18.901478,0.668919,0.648368,0.840219,0.606758,0.766732,0.673162,0.707048,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.697516,0.696333,0.690767,0.897520,0.710877,ok


grab one arch specific


KeyError: 'arch'

In [20]:
print("IMPORTANCES OLD PAPER")
print(len(df))
importancesOld =  [10, 10, 8, 8, 2, 5, 5]
df["adjacc"] = df["ok"]/df["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df[cols].values
df["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)

dfsorted = df.sort_values(["importanceold"], ascending = False)[0:20]

display(dfsorted)
# to subset some
# sub = df[((df["exp_desc"].str.contains
# ("A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse"))|(df["exp_desc"].str.contains("_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue")))]
# importanceold = sub.sort_values("importanceold", ascending=False)[0:20]
# display(importanceold)

IMPORTANCES OLD PAPER
2742


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg,adjacc,importanceold
2729,2729,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83838,7635,6362,14945,11081,42290,30489,38000,32268,4575,3638,100234,0.780288,0.849158,0.795191,0.741452,0.833268,0.720951,0.788004,resnet,0.788578,0.786075,0.785623,0.932887,0.794688
2721,2721,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.01}",107445,83151,7635,6415,14945,11094,42290,30862,38000,31120,4575,3660,99573,0.773894,0.818947,0.800000,0.742322,0.840210,0.729771,0.786250,resnet,0.784676,0.783161,0.781606,0.926735,0.793264
2272,2272,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83500,7635,6323,14945,11014,42290,30386,38000,32088,4575,3689,100014,0.777142,0.844421,0.806339,0.736969,0.828160,0.718515,0.786881,resnet,0.787051,0.784446,0.783691,0.930839,0.793247
2421,2421,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet,0.792305,0.790882,0.794901,0.950319,0.792402
2718,2718,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 'scale'}",107445,83975,7635,6295,14945,11157,42290,30521,38000,32436,4575,3566,100614,0.781563,0.853579,0.779454,0.746537,0.824492,0.721707,0.785154,resnet,0.786855,0.784256,0.784524,0.936423,0.792037
2722,2722,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}",107445,84035,7635,6287,14945,11159,42290,30537,38000,32490,4575,3562,100697,0.782121,0.855000,0.778579,0.746671,0.823445,0.722086,0.785156,resnet,0.787015,0.784397,0.784764,0.937196,0.792031
2409,2409,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86567,7635,6109,14945,11120,42290,32194,38000,33804,4575,3340,102087,0.805687,0.889579,0.730055,0.744062,0.800131,0.761267,0.785019,resnet,0.791541,0.790186,0.794082,0.950133,0.791783
1358,1358,A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83497,7635,6380,14945,10956,42290,30429,38000,32115,4575,3617,100104,0.777114,0.845132,0.790601,0.733088,0.835625,0.719532,0.784796,resnet,0.785087,0.782875,0.782332,0.931677,0.791741
2736,2736,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 100, 'gamma': 0.001}",107445,83426,7635,6334,14945,11059,42290,30763,38000,31631,4575,3639,99739,0.776453,0.832395,0.795410,0.739980,0.829601,0.727430,0.784963,resnet,0.784747,0.782835,0.782054,0.928280,0.791653
2710,2710,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 0.1, 'gamma': 'scale'}",107445,83098,7635,6403,14945,11149,42290,30522,38000,31417,4575,3607,99828,0.773400,0.826763,0.788415,0.746002,0.838638,0.721731,0.784310,resnet,0.783554,0.781582,0.780421,0.929108,0.791592


In [22]:

algs = []
for i in df["exp_details"]:
    if "svm" in i:
        algs.append("svm")
    else:
        algs.append("ok")

df["alg"] = algs

df_nonsvm = df[df["alg"]!="svm"]

print("IMPORTANCES OLD PAPER NO SVM")

importancesOld =  [10, 10, 8, 8, 2, 5, 5]
df_nonsvm["adjacc"] = df_nonsvm["ok"]/df_nonsvm["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df_nonsvm[cols].values
df_nonsvm["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)

dfsorted = df_nonsvm.sort_values(["importanceold"], ascending = False)[0:20]

display(dfsorted)

IMPORTANCES OLD PAPER NO SVM


/tmp/tmp.80hJAYorXg/ipykernel_4194109/1692613184.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonsvm["adjacc"] = df_nonsvm["ok"]/df_nonsvm["totalims"]
/tmp/tmp.80hJAYorXg/ipykernel_4194109/1692613184.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonsvm["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg,adjacc,importanceold,alg
2421,2421,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet,0.792305,0.790882,0.794901,0.950319,0.792402,ok
2409,2409,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86567,7635,6109,14945,11120,42290,32194,38000,33804,4575,3340,102087,0.805687,0.889579,0.730055,0.744062,0.800131,0.761267,0.785019,resnet,0.791541,0.790186,0.794082,0.950133,0.791783,ok
2439,2439,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86638,7635,6093,14945,11206,42290,32149,38000,33882,4575,3308,102096,0.806347,0.891632,0.723060,0.749816,0.798035,0.760203,0.784549,resnet,0.791511,0.789999,0.794136,0.950216,0.791384,ok
2637,2637,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86791,7635,6113,14945,11022,42290,32671,38000,33664,4575,3321,102212,0.807771,0.885895,0.725902,0.737504,0.800655,0.772547,0.784500,resnet,0.790930,0.790318,0.794401,0.951296,0.791381,ok
2625,2625,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86737,7635,6101,14945,11056,42290,32595,38000,33660,4575,3325,102212,0.807269,0.885789,0.726776,0.739779,0.799083,0.770750,0.784435,resnet,0.790909,0.790144,0.794204,0.951296,0.791276,ok
2493,2493,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86736,7635,6105,14945,11027,42290,32694,38000,33587,4575,3323,102152,0.807260,0.883868,0.726339,0.737839,0.799607,0.773091,0.784149,resnet,0.790501,0.789926,0.793970,0.950738,0.791007,ok
1507,1507,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86428,7635,6096,14945,11082,42290,32109,38000,33786,4575,3355,102029,0.804393,0.889105,0.733333,0.741519,0.798428,0.759258,0.784329,resnet,0.790777,0.789345,0.793166,0.949593,0.791001,ok
2457,2457,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86581,7635,6063,14945,11210,42290,32133,38000,33861,4575,3314,102095,0.805817,0.891079,0.724372,0.750084,0.794106,0.759825,0.783893,resnet,0.790961,0.789374,0.793557,0.950207,0.790621,ok
1988,1988,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86213,7635,6070,14945,11204,42290,31889,38000,33693,4575,3357,101954,0.802392,0.886658,0.733770,0.749682,0.795023,0.754055,0.783838,resnet,0.790304,0.788476,0.792178,0.948895,0.790482,ok
2427,2427,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86516,7635,6091,14945,11161,42290,32137,38000,33818,4575,3309,102065,0.805212,0.889947,0

In [24]:
# sorted to those just top 2 reran 7/15


algs = []
for i in df["exp_details"]:
    if "svm" in i:
        algs.append("svm")
    else:
        algs.append("ok")

df["alg"] = algs

df_nonsvm = df[df["alg"]!="svm"]

# df_nonsvm = df

print("IMPORTANCES OLD PAPER NO SVM")

importancesOld =  [10, 10, 8, 8, 2, 5, 5]
df_nonsvm["adjacc"] = df_nonsvm["ok"]/df_nonsvm["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df_nonsvm[cols].values
df_nonsvm["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)

sub = df_nonsvm[((df_nonsvm["exp_desc"].str.contains
("A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue"))|(df_nonsvm["exp_desc"].str.contains("_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse")))]
importanceold = sub.sort_values("importanceold", ascending=False)[0:20]
display(importanceold)



IMPORTANCES OLD PAPER NO SVM


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg,adjacc,importanceold,alg
2272,2272,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83500,7635,6323,14945,11014,42290,30386,38000,32088,4575,3689,100014,0.777142,0.844421,0.806339,0.736969,0.828160,0.718515,0.786881,resnet,0.787051,0.784446,0.783691,0.930839,0.793247,svm
2279,2279,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 100, 'gamma': 0.001}",107445,83069,7635,6336,14945,10995,42290,30734,38000,31317,4575,3687,99489,0.773130,0.824132,0.805902,0.735698,0.829862,0.726744,0.784467,resnet,0.783432,0.781633,0.780343,0.925953,0.791021,svm
2264,2264,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.01}",107445,82594,7635,6398,14945,11015,42290,30762,38000,30720,4575,3699,99163,0.768710,0.808421,0.808525,0.737036,0.837983,0.727406,0.783874,resnet,0.781495,0.780083,0.778026,0.922919,0.790758,svm
1988,1988,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86213,7635,6070,14945,11204,42290,31889,38000,33693,4575,3357,101954,0.802392,0.886658,0.733770,0.749682,0.795023,0.754055,0.783838,resnet,0.790304,0.788476,0.792178,0.948895,0.790482,ok
2261,2261,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 'scale'}",107445,83545,7635,6256,14945,11145,42290,30353,38000,32181,4575,3610,100338,0.777561,0.846868,0.789071,0.745734,0.819384,0.717735,0.783759,resnet,0.784997,0.782209,0.782105,0.933855,0.790388,svm
2265,2265,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}",107445,83615,7635,6242,14945,11171,42290,30391,38000,32217,4575,3594,100412,0.778212,0.847816,0.785574,0.747474,0.817551,0.718633,0.783410,resnet,0.784900,0.782110,0.782174,0.934543,0.790070,svm
1970,1970,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86265,7635,6051,14945,11143,42290,31992,38000,33718,4575,3361,101941,0.802876,0.887316,0.734645,0.745601,0.792534,0.756491,0.783317,resnet,0.789898,0.788207,0.792030,0.948774,0.789847,ok
1964,1964,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86215,7635,6042,14945,11132,42290,31999,38000,33677,4575,3365,101875,0.802411,0.886237,0.735519,0.744865,0.791356,0.756656,0.782926,resnet,0.789469,0.787798,0.791602,0.948160,0.789407,ok
2280,2280,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 100, 'gamma': 0.01}",107445,83691,7635,6254,14945,10995,42290,30072,38000,32775,4575,3595,100508,0.778919,0.862500,0.785792,0.735698,0.819122,0.711090,0.782840,resnet,0.784982,0.781860,0.782247,0.935437,0.789399,svm
1982,1982,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86201,7635,6058,14945,11127,42290,31960,38000,33707,4575,3349,101906,0.802280,0.887026,0.732022,0.744530,0.793451,0.755734,0.782553,resnet,0.789135,0.787485,0.791323,0.948448,0.789176,ok


In [42]:
# check mobilenet, maybe can justify "simplicity" w that arch, for operationalizing, if not too much diff

# sorted to those just top 2 reran 7/15


algs = []
for i in df["exp_details"]:
    if "svm" in i:
        algs.append("svm")
    else:
        algs.append("ok")

df["alg"] = algs

df_nonsvm = df[df["alg"]!="svm"]

# df_nonsvm = df

print("IMPORTANCES OLD PAPER NO SVM")

importancesOld =  [10, 10, 8, 8, 2, 5, 5]
df_nonsvm["adjacc"] = df_nonsvm["ok"]/df_nonsvm["totalims"]

cols = ["overall", "perc_snow_severe", "perc_poor_viz", "perc_snow", "adjacc", "perc_dry", "perc_wet"]
values = df_nonsvm[cols].values
df_nonsvm["importanceold"] = (values * importancesOld).sum(axis=1) / sum(importancesOld)
print(len(df_nonsvm))
# sub = df_nonsvm[(df_nonsvm["exp_desc"].str.contains("mobilenet"))]
# len(sub)
display(df_nonsvm.sort_values("importanceold", ascending=False)[0:20])


IMPORTANCES OLD PAPER NO SVM
100


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc,avgmetrics,weighted_avg_recall,impAndAvg,adjacc,importanceold,alg
51,107445,73284,7635,5984,14945,8968,42290,29816,38000,24788,4575,3728,96911,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_8_E75_AugTrue,2025-07-08 18:18:19.418131,0.682061,0.652316,0.814863,0.600067,0.783759,0.705037,0.711208,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703921,0.700835,0.698035,0.901959,0.720173,ok
47,107445,73875,7635,5928,14945,9094,42290,30082,38000,25120,4575,3651,97188,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_6_E75_AugTrue,2025-07-08 17:56:54.828344,0.687561,0.661053,0.798033,0.608498,0.776424,0.711327,0.711067,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705190,0.702101,0.700243,0.904537,0.720064,ok
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667000,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.705165,0.701636,0.700902,0.907292,0.718928,ok
75,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,4575,3800,96587,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue,2025-07-14 01:28:49.023233,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701169,0.698324,0.694460,0.898944,0.718561,ok
59,107445,71352,7635,5933,14945,9703,42290,28686,38000,23242,4575,3788,95100,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse,2025-07-08 18:44:48.704370,0.664079,0.611632,0.827978,0.649247,0.777079,0.678316,0.708851,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.697658,0.696185,0.689705,0.885104,0.717695,ok
55,107445,71623,7635,5955,14945,9605,42290,29051,38000,23253,4575,3759,95269,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-08 18:31:34.554699,0.666602,0.611921,0.821639,0.642690,0.779961,0.686947,0.708632,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.698124,0.696043,0.690425,0.886677,0.717666,ok
43,107445,73871,7635,5877,14945,9033,42290,30141,38000,25195,4575,3625,97259,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_4_E75_AugTrue,2025-07-08 17:35:31.599565,0.687524,0.663026,0.792350,0.604416,0.769745,0.712722,0.708452,_A_resnet_TRLETrue_ASTTrue_AugTrue,0.703220,0.700039,0.698671,0.905198,0.717416,ok
71,107445,72890,7635,5843,14945,9051,42290,29490,38000,24727,4575,3779,96679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugTrue,2025-07-14 00:59:47.014535,0.678394,0.650711,0.826011,0.605621,0.765291,0.697328,0.708992,_A_resnet_TRLETrue_ASTFalse_AugTrue,0.701343,0.699031,0.695472,0.899800,0.717285,ok
63,107445,70629,7635,5967,14945,9774,42290,28085,38000,22988,4575,3815,94838,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_6_E75_AugFalse,2025-07-08 18:58:01.654365,0.657350,0.604947,0.833880,0.653998,0.781532,0.664105,0.707693,_A_mobilenet_TRLETrue_ASTFalse_AugFalse,0.695107,0.694208,0.686417,0.882666,0.716718,ok
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.810710,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse,0.699260,0.696115,0.693324,0.900330,0.715544,ok


In [19]:
dd = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal_orig4.csv")
print(len(dd))

1828


In [35]:
# Individual conditions
resnet_cond = df['exp_details'].str.contains('A_resnet_TRLETrue_ASTFalse', na=False) & df['exp_details'].str.contains('AugTrue', na=False)

mobilenet_cond = df['exp_details'].str.contains('A_mobilenet_TRLETrue_ASTFalse', na=False) & df['exp_details'].str.contains('AugFalse', na=False)
print(len(df))
# Combine the conditions
exarch = df[resnet_cond | mobilenet_cond]
print(len(exarch))
importanceold_BL2 = exarch.sort_values("importanceold", ascending=False)
display(importanceold_BL2)

1828
0


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg,adjacc,importanceold
